In [11]:
import sys
sys.path.insert(0, '..')
import casadi as ca
import numpy as np
import sympy
from pyecca.symbolic import sympy_to_casadi

In [21]:
def sympy_derivations(poly_deg, cost_diff):
    t = sympy.symbols('t')
    p = sympy.Function('p')
    j, n = sympy.symbols('j, n', integer=True)
    
    # polynomial
    P = sympy.summation(p(j)*t**j, (j, 0, n))
    T = sympy.symbols('T')

    
    p = sympy.Matrix([p(i) for i in range(poly_deg + 1)])
    P = P.subs({n: poly_deg}).doit()
    
    cost_jerk = sympy.Matrix([sympy.integrate(P.diff(t, cost_diff)**2, (t, 0, T)).simplify()])
    Q = cost_jerk.jacobian(p).jacobian(p)
    
    T_ca = ca.SX.sym('T')
    Q_ca = sympy_to_casadi(f=Q, symbols={'T': T_ca})[0]
    f_Q = ca.Function('Q', [T_ca], [ca.sparsify(Q_ca)])
        
    return {
        'Q': Q,
        'f_Q': f_Q
    }
    
sym_exprs = sympy_derivations(poly_deg=5, cost_diff=3)
sym_exprs

{'Q': Matrix([
 [0, 0, 0,        0,        0,         0],
 [0, 0, 0,        0,        0,         0],
 [0, 0, 0,        0,        0,         0],
 [0, 0, 0,     72*T, 144*T**2,  240*T**3],
 [0, 0, 0, 144*T**2, 384*T**3,  720*T**4],
 [0, 0, 0, 240*T**3, 720*T**4, 1440*T**5]]),
 'f_Q': Function(Q:(i0)->(o0[6x6,9nz]) SXFunction)}

In [22]:
n_legs = 3
poly_deg = 5
cost_diff = 3

t = ca.SX.sym('t')  # current time
T = ca.SX.sym('T', n_legs)  # leg times
c = ca.SX.sym('c', poly_deg+1, n_legs)  # coefficients for all legs

p = ca.SX.zeros(n_legs)  # polynomial for each leg
t_start = 0  # start time
for j in range(n_legs):  # increment over all legs
    beta_j = (t - t_start)/T[j]  # scaled time
    t_start += T[j]  # start time
    for i in range(poly_deg):  # sum polynomial
        p[j] += c[i, j]*beta_j**i  # polynomial

sym_exprs = sympy_derivations(poly_deg=poly_deg, cost_diff=cost_diff)

J = 0
for j in range(n_legs):
    Q_j = sym_exprs['f_Q'](T[j])
    c_j = c[:, j]
    J += c_j.T@Q_j@c_j

J

SX(@1=72, @2=144, @3=240, @4=384, @5=720, @6=1440, ((((((((c_3*(@1*T_0))+(c_4*(@2*sq(T_0))))+(c_5*(@3*(T_0*sq(T_0)))))*c_3)+((((c_3*(@2*sq(T_0)))+(c_4*(@4*(T_0*sq(T_0)))))+(c_5*(@5*sq(sq(T_0)))))*c_4))+((((c_3*(@3*(T_0*sq(T_0))))+(c_4*(@5*sq(sq(T_0)))))+(c_5*(@6*(T_0*sq(sq(T_0))))))*c_5))+((((((c_9*(@1*T_1))+(c_10*(@2*sq(T_1))))+(c_11*(@3*(T_1*sq(T_1)))))*c_9)+((((c_9*(@2*sq(T_1)))+(c_10*(@4*(T_1*sq(T_1)))))+(c_11*(@5*sq(sq(T_1)))))*c_10))+((((c_9*(@3*(T_1*sq(T_1))))+(c_10*(@5*sq(sq(T_1)))))+(c_11*(@6*(T_1*sq(sq(T_1))))))*c_11)))+((((((c_15*(@1*T_2))+(c_16*(@2*sq(T_2))))+(c_17*(@3*(T_2*sq(T_2)))))*c_15)+((((c_15*(@2*sq(T_2)))+(c_16*(@4*(T_2*sq(T_2)))))+(c_17*(@5*sq(sq(T_2)))))*c_16))+((((c_15*(@3*(T_2*sq(T_2))))+(c_16*(@5*sq(sq(T_2)))))+(c_17*(@6*(T_2*sq(sq(T_2))))))*c_17))))

In [19]:
ca.jacobian(J, c)

SX(@1=240, @2=(@1*(T_0*sq(T_0))), @3=144, @4=(@3*sq(T_0)), @5=72, @6=(@5*T_0), @7=(@3*sq(T_0)), @8=(@1*(T_0*sq(T_0))), @9=720, @10=(@9*sq(sq(T_0))), @11=384, @12=(@11*(T_0*sq(T_0))), @13=(@9*sq(sq(T_0))), @14=1440, @15=(@14*(T_0*sq(sq(T_0)))), @16=(@1*(T_1*sq(T_1))), @17=(@3*sq(T_1)), @18=(@5*T_1), @19=(@3*sq(T_1)), @20=(@1*(T_1*sq(T_1))), @21=(@9*sq(sq(T_1))), @22=(@11*(T_1*sq(T_1))), @23=(@9*sq(sq(T_1))), @24=(@14*(T_1*sq(sq(T_1)))), @25=(@1*(T_2*sq(T_2))), @26=(@3*sq(T_2)), @27=(@5*T_2), @28=(@3*sq(T_2)), @29=(@1*(T_2*sq(T_2))), @30=(@9*sq(sq(T_2))), @31=(@11*(T_2*sq(T_2))), @32=(@9*sq(sq(T_2))), @33=(@14*(T_2*sq(sq(T_2)))), [[00, 00, 00, ((((@2*c_5)+(@4*c_4))+(((c_3*@6)+(c_4*@7))+(c_5*@8)))+(@6*c_3)), ((((@10*c_5)+(((c_3*@4)+(c_4*@12))+(c_5*@13)))+(@12*c_4))+(@7*c_3)), ((((((c_3*@2)+(c_4*@10))+(c_5*@15))+(@15*c_5))+(@13*c_4))+(@8*c_3)), 00, 00, 00, ((((@16*c_11)+(@17*c_10))+(((c_9*@18)+(c_10*@19))+(c_11*@20)))+(@18*c_9)), ((((@21*c_11)+(((c_9*@17)+(c_10*@22))+(c_11*@23)))+(@22*c_10

In [17]:
c.shape

(6, 3)